# Práctica 6: Planificación con incertidumbre (II)

## Inteligencia Artificial para la Robótica
### M.U en Lógica, Computación e Inteligencia Artificial
### Universidad de Sevilla

En esta práctica vamos a estudiar como definir y resolver POMDPs utilizando la biblioteca [pomdp-py](https://h2r.github.io/pomdp-py/html/)

* [Instala](https://h2r.github.io/pomdp-py/html/installation.html) la biblioteca y analiza la documentación existente en [pomdp-py](https://h2r.github.io/pomdp-py/html/)
* Prueba la biblioteca con los [ejemplos](https://h2r.github.io/pomdp-py/html/installation.html#test-things-out) incluidos

A continuación se incluye el código del ejemplo [Tiger](https://h2r.github.io/pomdp-py/html/examples.tiger.html)

* Analiza y prueba el código

In [ ]:
!pip install pomdp-py

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.0/5.0 MB 2.2 MB/s eta 0:00:00


In [ ]:
import pomdp_py
from pomdp_py.utils import TreeDebugger
import random
import numpy as np
import sys

In [ ]:
class TigerState(pomdp_py.State):
    def __init__(self, name):
        self.name = name
    def __hash__(self):
        return hash(self.name)
    def __eq__(self, other):
        if isinstance(other, TigerState):
            return self.name == other.name
        return False
    def __str__(self):
        return self.name
    def __repr__(self):
        return "TigerState(%s)" % self.name
    def other(self):
        if self.name.endswith("left"):
            return TigerState("tiger-right")
        else:
            return TigerState("tiger-left")

In [ ]:
class TigerAction(pomdp_py.Action):
    def __init__(self, name):
        self.name = name
    def __hash__(self):
        return hash(self.name)
    def __eq__(self, other):
        if isinstance(other, TigerAction):
            return self.name == other.name
        return False
    def __str__(self):
        return self.name
    def __repr__(self):
        return "TigerAction(%s)" % self.name

In [ ]:
class TigerObservation(pomdp_py.Observation):
    def __init__(self, name):
        self.name = name
    def __hash__(self):
        return hash(self.name)
    def __eq__(self, other):
        if isinstance(other, TigerObservation):
            return self.name == other.name
        return False
    def __str__(self):
        return self.name
    def __repr__(self):
        return "TigerObservation(%s)" % self.name

In [ ]:
# Observation model: definimos las matrices de transicion
class ObservationModel(pomdp_py.ObservationModel):
    def __init__(self, noise=0.15):
        self.noise = noise

    def probability(self, observation, next_state, action):
        if action.name == "listen":
            # heard the correct growl
            if observation.name == next_state.name:
                return 1.0 - self.noise
            else:
                return self.noise
        else:
            return 0.5

    def sample(self, next_state, action):
        if action.name == "listen":
            thresh = 1.0 - self.noise
        else:
            thresh = 0.5

        if random.uniform(0,1) < thresh:
            return TigerObservation(next_state.name)
        else:
            return TigerObservation(next_state.other().name)

    def get_all_observations(self):
        """Only need to implement this if you're using
        a solver that needs to enumerate over the observation space
        (e.g. value iteration)"""
        return [TigerObservation(s)
                for s in {"tiger-left", "tiger-right"}]

In [ ]:
# Transition Model: me dice que estado alcanzo al ejecutar una accion: T:SxAxS y devuelve probabilidad
class TransitionModel(pomdp_py.TransitionModel):

    # probabilidad de alcanzar el estado, estando en un estado y haciendo una accion
    # esto es lo que se implementa en el modelo
    def probability(self, next_state, state, action):
        """According to problem spec, the world resets once
        action is open-left/open-right. Otherwise, stays the same"""
        if action.name.startswith("open"):
            return 0.5 # siempre devuelve una probabilidad (el tigre sigue siempre en el mismo sitio)
        else: # nunca llega a ser 0 o 1.
            if next_state.name == state.name:
                return 1.0 - 1e-9
            else:
                return 1e-9


    def sample(self, state, action):
        if action.name.startswith("open"):
            return random.choice(self.get_all_states())
        else:
            return TigerState(state.name)

    def get_all_states(self):
        """Only need to implement this if you're using
        a solver that needs to enumerate over the observation space (e.g. value iteration)"""
        return [TigerState(s) for s in {"tiger-left", "tiger-right"}]

In [ ]:
# Reward Model: SxA es la función recompensa (hay que implementarlo)
# reward asociado al estado
class RewardModel(pomdp_py.RewardModel):
    def _reward_func(self, state, action):
        if action.name == "open-left":
            if state.name == "tiger-right":
                return 10
            else:
                return -100
        elif action.name == "open-right":
            if state.name == "tiger-left":
                return 10
            else:
                return -100
        else: # listen
            return -1

    # porque el reward puede ser no determinista
    # estoy en un estado, ejecuto accion y llego a otro estado, devuelve el reward.
    def sample(self, state, action, next_state):
        # deterministic
        return self._reward_func(state, action) # devuelve el reward de aplicar al estado una accion

In [ ]:
# Policy Model
class PolicyModel(pomdp_py.RolloutPolicy):
    """A simple policy model with uniform prior over a
       small, finite action space"""

    ACTIONS = [TigerAction(s)
              for s in {"open-left", "open-right", "listen"}]


    def sample(self, state): # dame una accion al azar aplicable al estado
        return random.sample(self.get_all_actions(), 1)[0]


    # lo mismo que sample: función que ejecuta acciones hasta llegar a un estado de parada.
    def rollout(self, state, history=None):
        """Treating this PolicyModel as a rollout policy"""
        return self.sample(state)

    def get_all_actions(self, state=None, history=None): # tienen que estar todas las acciones
    # tenemos todas las acciones a partir de un etsdo concreto.
    # siempre devuelvo la misma lista
        return PolicyModel.ACTIONS

In [ ]:
class TigerProblem(pomdp_py.POMDP):
    """
    In fact, creating a TigerProblem class is entirely optional
    to simulate and solve POMDPs. But this is just an example
    of how such a class can be created.
    """

    def __init__(self, obs_noise, init_true_state, init_belief):
        """init_belief is a Distribution."""
        # separamos el agente del entorno
        agent = pomdp_py.Agent(init_belief,
                               PolicyModel(),
                               TransitionModel(),
                               ObservationModel(obs_noise),
                               RewardModel())
        env = pomdp_py.Environment(init_true_state, # le pasamos cual es el estado real (instancia de TigerState)
                                   TransitionModel(),
                                   RewardModel())
        super().__init__(agent, env, name="TigerProblem")

    @staticmethod
    def create(state="tiger-left", belief=0.5, obs_noise=0.15):
        """
        Args:
            state (str): could be 'tiger-left' or 'tiger-right';
                         True state of the environment
            belief (float): Initial belief that the target is
                            on the left; Between 0-1.
            obs_noise (float): Noise for the observation
                               model (default 0.15)
        """
        init_true_state = TigerState(state)
        # crea el belief (distr de prob) con un histograma -> diccionario
        # cada clave es un estado
        init_belief = pomdp_py.Histogram({
            TigerState("tiger-left"): belief,
            TigerState("tiger-right"): 1.0 - belief
        })
        tiger_problem = TigerProblem(obs_noise,
                                     init_true_state, init_belief)
        tiger_problem.agent.set_belief(init_belief, prior=True)
        return tiger_problem

In [ ]:
def test_planner(tiger_problem, planner, nsteps=3,
                 debug_tree=False):
    """
    Runs the action-feedback loop of Tiger problem POMDP

    Args:
        tiger_problem (TigerProblem): a problem instance
        planner (Planner): a planner
        nsteps (int): Maximum number of steps to run this loop.
        debug_tree (bool): True if get into the pdb with a
                           TreeDebugger created as 'dd' variable.
    """
    for i in range(nsteps):
        action = planner.plan(tiger_problem.agent) # método plan integrado y pasamos el agente (planificación)
        if debug_tree:
            from pomdp_py.utils import TreeDebugger
            dd = TreeDebugger(tiger_problem.agent.tree)
            import pdb; pdb.set_trace()

        print("==== Step %d ====" % (i+1))
        print("True state:", tiger_problem.env.state)
        print("Belief:", tiger_problem.agent.cur_belief)
        print("Action:", action)
        # There is no state transition for the tiger domain.
        # In general, the ennvironment state can be transitioned
        # using
        #
        #   reward = tiger_problem.env.state_transition(action, execute=True)
        #
        # Or, it is possible that you don't have control
        # over the environment change (e.g. robot acting
        # in real world); In that case, you could skip
        # the state transition and re-estimate the state
        # (e.g. through the perception stack on the robot).
        reward = tiger_problem.env.reward_model.sample(tiger_problem.env.state, action, None)
        print("Reward:", reward)

        # Let's create some simulated real observation;
        # Here, we use observation based on true state for sanity
        # checking solver behavior. In general, this observation
        # should be sampled from agent's observation model, as
        #
        #    real_observation = tiger_problem.agent.observation_model.sample(tiger_problem.env.state, action)
        #
        # or coming from an external source (e.g. robot sensor
        # reading). Note that tiger_problem.env.state stores the
        # environment state after action execution.
        real_observation = TigerObservation(tiger_problem.env.state.name) # ejecucion
        print(">> Observation:",  real_observation)
        tiger_problem.agent.update_history(action, real_observation) # actualizar la historia

        # Update the belief. If the planner is POMCP, planner.update
        # also automatically updates agent belief.
        planner.update(tiger_problem.agent, action, real_observation) # actualizar el planificador
        if isinstance(planner, pomdp_py.POUCT):
            print("Num sims:", planner.last_num_sims)
            print("Plan time: %.5f" % planner.last_planning_time)

        if isinstance(tiger_problem.agent.cur_belief,
                      pomdp_py.Histogram):
            new_belief = pomdp_py.update_histogram_belief(
                tiger_problem.agent.cur_belief,
                action, real_observation,
                tiger_problem.agent.observation_model,
                tiger_problem.agent.transition_model)
            tiger_problem.agent.set_belief(new_belief)

        if action.name.startswith("open"):
            # Make it clearer to see what actions are taken
            # until every time door is opened.
            print("\n")

In [ ]:
def main():
    init_true_state = random.choice([TigerState("tiger-left"),
                                     TigerState("tiger-right")])
    init_belief = pomdp_py.Histogram({TigerState("tiger-left"): 0.5,
                                      TigerState("tiger-right"): 0.5})
    tiger_problem = TigerProblem(0.15,  # observation noise
                                 init_true_state, init_belief)

    print("** Testing value iteration **")
    vi = pomdp_py.ValueIteration(horizon=3, discount_factor=0.95)
    test_planner(tiger_problem, vi, nsteps=3)

    # Reset agent belief
    tiger_problem.agent.set_belief(init_belief, prior=True)

    print("\n** Testing POUCT **")
    pouct = pomdp_py.POUCT(max_depth=3, discount_factor=0.95,
                           num_sims=4096, exploration_const=50,
                           rollout_policy=tiger_problem.agent.policy_model,
                           show_progress=True)
    test_planner(tiger_problem, pouct, nsteps=10)
    TreeDebugger(tiger_problem.agent.tree).pp

    # Reset agent belief
    tiger_problem.agent.set_belief(init_belief, prior=True)
    tiger_problem.agent.tree = None

    print("** Testing POMCP **")
    tiger_problem.agent.set_belief(pomdp_py.Particles.from_histogram(init_belief, num_particles=100), prior=True)
    pomcp = pomdp_py.POMCP(max_depth=3, discount_factor=0.95, # instanciar el POMCP funciona por muestreo
                           num_sims=1000, exploration_const=50,
                           rollout_policy=tiger_problem.agent.policy_model,
                           show_progress=True, pbar_update_interval=500)
    test_planner(tiger_problem, pomcp, nsteps=10)
    TreeDebugger(tiger_problem.agent.tree).pp

In [ ]:
main()

** Testing value iteration **
==== Step 1 ====
True state: tiger-left
Belief: {TigerState(tiger-left): 0.5, TigerState(tiger-right): 0.5}
Action: listen
Reward: -1
>> Observation: tiger-left
==== Step 2 ====
True state: tiger-left
Belief: {TigerState(tiger-left): 0.85, TigerState(tiger-right): 0.15}
Action: listen
Reward: -1
>> Observation: tiger-left
==== Step 3 ====
True state: tiger-left
Belief: {TigerState(tiger-left): 0.9697986575573173, TigerState(tiger-right): 0.03020134244268276}
Action: listen
Reward: -1
>> Observation: tiger-left

** Testing POUCT **


100%|█████████▉| 4095/4096 [00:01<00:00, 2652.49it/s]


==== Step 1 ====
True state: tiger-left
Belief: {TigerState(tiger-left): 0.5, TigerState(tiger-right): 0.5}
Action: listen
Reward: -1
>> Observation: tiger-left
Num sims: 4096
Plan time: 1.54372


100%|█████████▉| 4095/4096 [00:01<00:00, 2682.32it/s]


==== Step 2 ====
True state: tiger-left
Belief: {TigerState(tiger-left): 0.85, TigerState(tiger-right): 0.15}
Action: listen
Reward: -1
>> Observation: tiger-left
Num sims: 4096
Plan time: 1.52657


...

==== Step 8 ====
True state: tiger-left
Belief: [(TigerState(tiger-right), 0.47835051546391755), (TigerState(tiger-left), 0.5216494845360825)]
Action: listen
Reward: -1
>> Observation: tiger-left
Num sims: 1000
Plan time: 0.18071


100%|██████████| 1000/1000 [00:00<00:00, 7514.44it/s]


==== Step 9 ====
True state: tiger-left
Belief: [(TigerState(tiger-right), 0.12133072407045009), (TigerState(tiger-left), 0.8786692759295499)]
Action: listen
Reward: -1
>> Observation: tiger-left
Num sims: 1000
Plan time: 0.12702


100%|██████████| 1000/1000 [00:00<00:00, 7677.83it/s]

==== Step 10 ====
True state: tiger-left
Belief: [(TigerState(tiger-left), 0.9749009247027741), (TigerState(tiger-right), 0.02509907529722589)]
Action: open-right
Reward: 10
>> Observation: tiger-left
Num sims: 1000
Plan time: 0.12843


_VNodePP(n=950, v=-2.331)(depth=0)
├─── ₀listen⟶_QNodePP(n=933, v=-2.331)
│    ├─── ₀tiger-left⟶_VNodePP(n=414, v=0.141)(depth=1)
│    │    ├─── ₀listen⟶_QNodePP(n=378, v=0.141)
│    │    │    ├─── ₀tiger-left⟶_VNodePP(n=172, v=6.589)(depth=2)
│    │    │    │    ├─── ₀listen⟶_QNodePP(n=42, v=-1.000)
│    │    │    │    └─── ₂open-right⟶_QNodePP(n=129, v=6.589)
│    │    │    └─── ₁tiger-right⟶_VNodePP(n=56, v=-1.000)(depth=2)
│    │    │         ├─── ₀listen⟶_QNodePP(n=54, v=-1.000)
│    │    ├─── ₁open-left⟶_QNodePP(n=2, v=-100.000)
│    │    └─── ₂open-right⟶_QNodePP(n=34, v=-15.028)
│    │         ├─── ₀tiger-left⟶_VNodePP(n=2, v=0.000)(depth=2)
│    └─── ₁tiger-right⟶_VNodePP(n=411, v=0.348)(depth=1)
│         ├─── ₀listen⟶_QNodePP(n=265, v=0.348)


## Ejercicio

* Analiza la página web http://pomdp.org/
* Elije un ejemplo sencillo de http://pomdp.org/examples/
* Define el POMDP elegido utilizando lo aprendido de [pomdp-py](https://h2r.github.io/pomdp-py/html/)
* Resuelve el POMDP utilizando diversos planificadores
* Comenta los resultados



# Michael's 1D maze

discount: 0.75

values: reward

states: left middle right goal

actions: w0 e0

observations: nothing goal

T: w0

1.0 0.0 0.0 0.0

1.0 0.0 0.0 0.0

0.0 0.0 0.0 1.0

0.333333 0.333333 0.333333 0.0

T: e0

0.0 1.0 0.0 0.0

0.0 0.0 0.0 1.0

0.0 0.0 1.0 0.0

0.333333 0.333333 0.333333 0.0

O: *

1.0 0.0

1.0 0.0

1.0 0.0

0.0 1.0

R: * : * : goal : goal 1.0

## Dominio

En primer lugar, se define el conjunto de estados, las acciones y las observaciones.

In [ ]:
class RobotState(pomdp_py.State):
    def __init__(self, name):
        self.name = name
    def __hash__(self):
        return hash(self.name)
    def __eq__(self, other):
        if isinstance(other, RobotState):
            return self.name == other.name
        return False
    def __str__(self):
        return self.name
    def __repr__(self):
        return "RobotState(%s)" % self.name

    def other(self):
        numero = np.random.randint(3)
        if self.name == "left":
            match numero:
                case 0:
                    return RobotState("middle")
                case 1:
                    return RobotState("right")
                case 2:
                    return RobotState("goal")

        if self.name == "right":
            match numero:
                case 0:
                    return RobotState("left")
                case 1:
                    return RobotState("middle")
                case 2:
                    return RobotState("goal")

        if self.name == "middle":
            match numero:
                case 0:
                    return RobotState("left")
                case 1:
                    return RobotState("right")
                case 2:
                    return RobotState("goal")
        else:
            match numero:
                case 0:
                    return RobotState("left")
                case 1:
                    return RobotState("middle")
                case 2:
                    return RobotState("right")



In [ ]:
class RobotAction(pomdp_py.Action):
    def __init__(self, name):
        self.name = name
    def __hash__(self):
        return hash(self.name)
    def __eq__(self, other):
        if isinstance(other, RobotAction):
            return self.name == other.name
        return False
    def __str__(self):
        return self.name
    def __repr__(self):
        return "RobotAction(%s)" % self.name

In [ ]:
class RobotObservation(pomdp_py.Observation):
    def __init__(self, name):
        self.name = name
    def __hash__(self):
        return hash(self.name)
    def __eq__(self, other):
        if isinstance(other, RobotObservation):
            return self.name == other.name
        return False
    def __str__(self):
        return self.name
    def __repr__(self):
        return "RobotObservation(%s)" % self.name

In [ ]:
# Observation model
class ObservationModel(pomdp_py.ObservationModel):

    def __init__(self, noise=0.15): # ruido de la observación
        self.noise = noise


    def probability(self, observation, next_state, action):

        ''' O: *
                    nothing goal
            left     1.0   0.0
            middle   1.0   0.0
            right    1.0   0.0
            goal     0.0   1.0
        '''

        # si la observación es 'nothing' y el estado siguiente no es 'goal'
        if observation.name == "nothing" and next_state.name != "goal":
            return 1.0 - self.noise
        # si la observación y el siguiente estado son el objetivo
        elif observation.name == "goal" and next_state.name == "goal":
            return 1.0 - self.noise
        else:
            return self.noise

    # muestrear una observación de acuerdo a la probabilidad: a partir de la acción realizada y el estado alcanzado, devuelve la observacion
    def sample(self, next_state, action):
        if action.name == "w0" or action.name == "e0":
            thresh = 1.0 - self.noise
        else:
            thresh = 0.5

        if random.uniform(0,1) < thresh:
            return RobotObservation(next_state.name) # devuelve el estado correcto
        else:
            return RobotObservation(next_state.other().name) # si no se devuelve otro de los estados

    def get_all_observations(self):
        # devuelve una lista con todas las observaciones
        return [RobotObservation(s) for s in {"nothing", "goal"}]

In [ ]:
# Transition Model:
class TransitionModel(pomdp_py.TransitionModel):

    ''' Devuelve la probabilidad de alcanzar un estado al ejecutar una acción en otro estado.'''
    # esto es lo que se implementa en el modelo
    def probability(self, next_state, state, action):

        ''' T: w0
            1.0 0.0 0.0 0.0
            1.0 0.0 0.0 0.0
            0.0 0.0 0.0 1.0
            0.333333 0.333333 0.333333 0.0

            T: e0
            0.0 1.0 0.0 0.0
            0.0 0.0 0.0 1.0
            0.0 0.0 1.0 0.0
            0.333333 0.333333 0.333333 0.0
        '''

        if action.name == "w0":
            if state.name == "left" and next_state.name == "left":
                return 1.0 - 1e-9
            elif state.name == "middle" and next_state.name == "left":
                return 1.0 - 1e-9
            elif state.name == "right" and next_state.name == "goal":
                return 1.0 - 1e-9
            elif state.name == "goal" and next_state.name != "goal":
                return 0.333333
            else:
                return 1e-9
        else:
            if state.name == "left" and next_state.name == "middle":
                return 1.0 - 1e-9
            elif state.name == "middle" and next_state.name == "goal":
                return 1.0 - 1e-9
            elif state.name == "right" and next_state.name == "right":
                return 1.0 - 1e-9
            elif state.name == "goal" and next_state.name != "goal":
                return 0.333333
            else:
                return 1e-9

    # muestrear: a partir de la acción realizada y el estado en el que está, devuelve un estado al azar
    def sample(self, state, action):
        if action.name == "w0":
            match state.name:
                case "left":
                    return RobotState("left")
                case "middle":
                    return RobotState("left")
                case "right":
                    return RobotState("goal")
                case "goal":
                    numero = np.random.randint(3)
                    return self.get_all_states()[numero]
        elif action.name == "e0":
            match state.name:
                case "left":
                    return RobotState("middle")
                case "middle":
                    return RobotState("goal")
                case "right":
                    return RobotState("right")
                case "goal":
                    numero = np.random.randint(3)
                    return self.get_all_states()[numero]

    def get_all_states(self):
        """Only need to implement this if you're using
        a solver that needs to enumerate over the observation space (e.g. value iteration)"""
        return [RobotState(s) for s in {"left", "middle", "right", "goal"}]

In [ ]:
# Reward Model

class RewardModel(pomdp_py.RewardModel):
    ''' SxA es la función recompensa asociada a cada estado '''

    def _reward_func(self, state, action):
      ''' R: * : * : goal : goal 1.0 '''
      if action.name == "w0" or action.name == "e0":
            if state.name == "goal":
                return 1
            else:
                return 0

    def sample(self, state, action, next_state):
        return self._reward_func(state, action) # devuelve el reward de aplicar al estado una accion

In [ ]:
# Policy Model
class PolicyModel(pomdp_py.RolloutPolicy):
    """A simple policy model with uniform prior over a
       small, finite action space"""

    ACTIONS = [RobotAction(s) for s in {"w0", "e0"}]


    def sample(self, state): # dame una accion al azar aplicable al estado
        return random.sample(self.get_all_actions(), 1)[0]


    # función que ejecuta acciones hasta llegar a un estado de parada.
    def rollout(self, state, history=None):
        """Treating this PolicyModel as a rollout policy"""
        return self.sample(state)

    def get_all_actions(self, state=None, history=None):
    # lista de todas las acciones a partir de un estado concreto.
        return PolicyModel.ACTIONS

In [ ]:
class MazeProblem(pomdp_py.POMDP):
    """
    Creación del problema del laberinto.
    """

    def __init__(self, obs_noise, init_true_state, init_belief):
        """init_belief is a Distribution."""
        # separamos el agente del entorno
        agent = pomdp_py.Agent(init_belief,
                               PolicyModel(),
                               TransitionModel(),
                               ObservationModel(obs_noise),
                               RewardModel())
        env = pomdp_py.Environment(init_true_state, # le pasamos cual es el estado real
                                   TransitionModel(),
                                   RewardModel())
        super().__init__(agent, env, name="MazeProblem")

    @staticmethod
    def create(state="left", belief=0.25, obs_noise=0.0):
        """
        Args:
            state (str): could be 'left', 'middle', 'right' or 'goal';
                         True state of the environment
            belief (float): Initial belief that the target is
                            on the left; Between 0-1.
            obs_noise (float): Noise for the observation
                               model
        """
        init_true_state = RobotState(state)
        # crea el belief (distribucion de probabilidad)
        # cada clave es un estado
        init_belief = pomdp_py.Histogram({
            RobotState("left"): belief,
            RobotState("middle"): belief,
            RobotState("right"): belief,
            RobotState("goal"): belief
        })
        maze_problem = MazeProblem(obs_noise,
                                     init_true_state, init_belief)
        maze_problem.agent.set_belief(init_belief, prior=True)
        return maze_problem

In [ ]:
def test_planner2(maze_problem, planner, nsteps=3,
                 debug_tree=False):
    """

    Args:
        maze_problem (MazeProblem): a problem instance
        planner (Planner): a planner
        nsteps (int): Maximum number of steps to run this loop.
        debug_tree (bool): True if get into the pdb with a
                           TreeDebugger created as 'dd' variable.
    """
    for i in range(nsteps):

        action = planner.plan(maze_problem.agent) # método plan integrado y pasamos el agente (planificación)
        if debug_tree:
            from pomdp_py.utils import TreeDebugger
            dd = TreeDebugger(maze_problem.agent.tree)
            import pdb; pdb.set_trace()

        print("==== Step %d ====" % (i+1))
        print("True state:", maze_problem.env.state)
        print("Belief:", maze_problem.agent.cur_belief)
        print("Action:", action)

        reward = maze_problem.env.reward_model.sample(maze_problem.env.state, action, None)
        print("Reward:", reward)

        if maze_problem.env.state.name != "goal":
            real_observation = RobotObservation("nothing")
        else:
            real_observation = RobotObservation("goal")

        print(">> Observation:",  real_observation)
        maze_problem.agent.update_history(action, real_observation) # actualizar la historia

        # Update the belief. If the planner is POMCP, planner.update
        # also automatically updates agent belief.
        planner.update(maze_problem.agent, action, real_observation) # actualizar el planificador
        if isinstance(planner, pomdp_py.POUCT):
            print("Num sims:", planner.last_num_sims)
            print("Plan time: %.5f" % planner.last_planning_time)

        if isinstance(maze_problem.agent.cur_belief,
                      pomdp_py.Histogram):
            new_belief = pomdp_py.update_histogram_belief(
                maze_problem.agent.cur_belief,
                action, real_observation,
                maze_problem.agent.observation_model,
                maze_problem.agent.transition_model)
            maze_problem.agent.set_belief(new_belief)



In [ ]:
def main():
    init_true_state = random.choice([RobotState("left"),
                                     RobotState("middle"),
                                     RobotState("right"),
                                     RobotState("goal")])
    init_belief = pomdp_py.Histogram({RobotState("left"): 0.25,
                                      RobotState("middle"): 0.25,
                                      RobotState("right"): 0.25,
                                      RobotState("goal"): 0.25})
    maze_problem = MazeProblem(0.15,  # observation noise
                                 init_true_state, init_belief)

    print("** Testing value iteration **")
    vi = pomdp_py.ValueIteration(horizon=3, discount_factor=0.75)
    test_planner2(maze_problem, vi, nsteps=3)

    # Reset agent belief
    maze_problem.agent.set_belief(init_belief, prior=True)

    print("\n** Testing POUCT **")
    pouct = pomdp_py.POUCT(max_depth=3, discount_factor=0.75,
                           num_sims=4096, exploration_const=50,
                           rollout_policy=maze_problem.agent.policy_model,
                           show_progress=True)
    test_planner2(maze_problem, pouct, nsteps=10)
    #TreeDebugger(maze_problem.agent.tree).pp

    # Reset agent belief
    maze_problem.agent.set_belief(init_belief, prior=True)
    maze_problem.agent.tree = None



In [ ]:
main()

** Testing value iteration **
==== Step 1 ====
True state: middle
Belief: {RobotState(left): 0.25, RobotState(middle): 0.25, RobotState(right): 0.25, RobotState(goal): 0.25}
Action: e0
Reward: 0
>> Observation: nothing
==== Step 2 ====
True state: middle
Belief: {RobotState(left): 0.10493820047037097, RobotState(middle): 0.41975311323343323, RobotState(right): 0.41975311323343323, RobotState(goal): 0.055555573062762695}
Action: e0
Reward: 0
>> Observation: nothing
==== Step 3 ====
True state: middle
Belief: {RobotState(left): 0.028301872132445612, RobotState(middle): 0.18867914017227913, RobotState(right): 0.6698114201316056, RobotState(goal): 0.11320756756366976}
Action: w0
Reward: 0
>> Observation: nothing

** Testing POUCT **


100%|█████████▉| 4095/4096 [00:01<00:00, 2737.99it/s]


==== Step 1 ====
True state: middle
Belief: {RobotState(left): 0.25, RobotState(middle): 0.25, RobotState(right): 0.25, RobotState(goal): 0.25}
Action: e0
Reward: 0
>> Observation: nothing
Num sims: 4096
Plan time: 1.49552


100%|█████████▉| 4095/4096 [00:01<00:00, 3020.14it/s]


==== Step 2 ====
True state: middle
Belief: {RobotState(left): 0.10493820047037097, RobotState(middle): 0.41975311323343323, RobotState(right): 0.41975311323343323, RobotState(goal): 0.055555573062762695}
Action: e0
Reward: 0
>> Observation: nothing
Num sims: 4096
Plan time: 1.35578


...

==== Step 8 ====
True state: middle
Belief: {RobotState(left): 0.04888001773354327, RobotState(middle): 0.68910615379413, RobotState(right): 0.23004375668913007, RobotState(goal): 0.03197007178319658}
Action: e0
Reward: 0
>> Observation: nothing
Num sims: 4096
Plan time: 1.62633


100%|█████████▉| 4095/4096 [00:01<00:00, 2289.03it/s]


==== Step 9 ====
True state: middle
Belief: {RobotState(left): 0.024639679925390012, RobotState(middle): 0.13765685665671198, RobotState(right): 0.5565317927187788, RobotState(goal): 0.281171670699119}
Action: w0
Reward: 0
>> Observation: nothing
Num sims: 4096
Plan time: 1.78885


100%|█████████▉| 4095/4096 [00:02<00:00, 1763.55it/s]

==== Step 10 ====
True state: middle
Belief: {RobotState(left): 0.47264179453044924, RobotState(middle): 0.17302447469294668, RobotState(right): 0.17302447469294668, RobotState(goal): 0.18130925608365742}
Action: e0
Reward: 0
>> Observation: nothing
Num sims: 4096
Plan time: 2.32191
